In [ ]:
import pandas as pd
import string
import os
import numpy as np
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image
import urllib
import requests
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import roc_curve, auc, classification_report
from sklearn.preprocessing import label_binarize
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
import time
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import operator
import folium
from itertools import cycle, islice
from pandas import options
import warnings
import pickle
import re
import nltk
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
from matplotlib.pyplot import figure
from nltk.corpus import stopwords 
import nltk


nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

#Preprocessing

##Load dataframes

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = "/content/drive/My Drive/"

#Read data from csv file 
df = pd.read_csv(path + 'vaccine_train_set.csv',nrows=15976) 
df = df.dropna()
df

,Unnamed: 0,tweet,label
0,0,Sip N Shop Come thru right now #Marjais #Popul...,0
1,1,I don't know about you but My family and I wil...,1
2,2,@MSignorile Immunizations should be mandatory....,2
3,3,President Obama spoke in favor of vaccination ...,0
4,4,"""@myfoxla: Arizona monitoring hundreds for mea...",0
...,...,...,...
15971,15971,@Salon if u believe the anti-vax nutcases caus...,1
15972,15972,How do you feel about parents who don't #vacci...,0
15973,15973,70 Preschoolers Tested for Measles in Simi Val...,0
15974,15974,Finance Minister: Budget offers room to procur...,0


In [ ]:
path = "/content/drive/My Drive/"

#Read data from csv file 
df2 = pd.read_csv(path + 'vaccine_validation_set.csv',nrows=2282) 
df2 = df2.dropna()
df2

,Unnamed: 0,tweet,label
0,0,@user They had a massive surge in with covid d...,1
1,1,Required vaccines for school: Parents and guar...,0
2,2,“@KCStar: Two more Johnson County children hav...,0
3,3,NV can do better. Which states are the best (a...,2
4,4,Nothing like killing ourselves w/ our own fear...,2
...,...,...,...
2277,2277,RT @abc7: Number of measles cases reported in ...,0
2278,2278,"Evidence points to the idea that ""measles affe...",0
2279,2279,"Where's @SavedYouAClick ""@voxdotcom: Why you s...",2
2280,2280,Some of my favorite people have autism. If tha...,2


In [ ]:
df2 = df2.drop(['Unnamed: 0'], axis=1)
df2

,tweet,label
0,@user They had a massive surge in with covid d...,1
1,Required vaccines for school: Parents and guar...,0
2,“@KCStar: Two more Johnson County children hav...,0
3,NV can do better. Which states are the best (a...,2
4,Nothing like killing ourselves w/ our own fear...,2
...,...,...
2277,RT @abc7: Number of measles cases reported in ...,0
2278,"Evidence points to the idea that ""measles affe...",0
2279,"Where's @SavedYouAClick ""@voxdotcom: Why you s...",2
2280,Some of my favorite people have autism. If tha...,2


In [ ]:
df = df.drop(['Unnamed: 0'], axis=1)
df

,tweet,label
0,Sip N Shop Come thru right now #Marjais #Popul...,0
1,I don't know about you but My family and I wil...,1
2,@MSignorile Immunizations should be mandatory....,2
3,President Obama spoke in favor of vaccination ...,0
4,"""@myfoxla: Arizona monitoring hundreds for mea...",0
...,...,...
15971,@Salon if u believe the anti-vax nutcases caus...,1
15972,How do you feel about parents who don't #vacci...,0
15973,70 Preschoolers Tested for Measles in Simi Val...,0
15974,Finance Minister: Budget offers room to procur...,0


##Remove unneccesary characters for train and validation set

In [ ]:
df['tweet'] = df['tweet'].str.lower()
#Define function for removing special characters
def remove_special_characters(tweet, remove_digits=True):
    tweet=re.sub(r'[^a-zA-z0-9\s]','',tweet)
    return tweet
df['tweet'] = df['tweet'].apply(lambda x:''.join([i for i in x if i not in string.punctuation]))
#Apply function on review column
df['tweet']=df['tweet'].apply(remove_special_characters)


df

,tweet,label
0,sip n shop come thru right now marjais popular...,0
1,i dont know about you but my family and i will...,1
2,msignorile immunizations should be mandatory p...,2
3,president obama spoke in favor of vaccination ...,0
4,myfoxla arizona monitoring hundreds for measle...,0
...,...,...
15971,salon if u believe the antivax nutcases caused...,1
15972,how do you feel about parents who dont vaccina...,0
15973,70 preschoolers tested for measles in simi val...,0
15974,finance minister budget offers room to procure...,0


In [ ]:
df2['tweet'] = df2['tweet'].str.lower()
#Define function for removing special characters
def remove_special_characters(tweet, remove_digits=True):
    tweet=re.sub(r'[^a-zA-z0-9\s]','',tweet)
    return tweet
df2['tweet'] = df2['tweet'].apply(lambda x:''.join([i for i in x if i not in string.punctuation]))
#Apply function on review column
df2['tweet']=df2['tweet'].apply(remove_special_characters)


df2

,tweet,label
0,user they had a massive surge in with covid de...,1
1,required vaccines for school parents and guard...,0
2,kcstar two more johnson county children have m...,0
3,nv can do better which states are the best and...,2
4,nothing like killing ourselves w our own fear ...,2
...,...,...
2277,rt abc7 number of measles cases reported in ca...,0
2278,evidence points to the idea that measles affec...,0
2279,wheres savedyouaclick voxdotcom why you should...,2
2280,some of my favorite people have autism if that...,2


In [ ]:
df['label'].value_counts()

0    7458
2    6445
1    2073
Name: label, dtype: int64

In [ ]:
df2['label'].value_counts()

0    1065
2     921
1     296
Name: label, dtype: int64

##Remove links

In [ ]:
import re
#Remove tags and links 
tag = re.compile(r'<[^>]+>')

df['tweet'] = df['tweet'].apply(lambda x: tag.sub('', x)) #removing html labels

df['tweet'] = df['tweet'].replace(r'http\S+', '', regex=True).replace(r'www.\S+', '', regex=True).replace(r'http\S+', '', regex=True).replace(r'"', '', regex=True)

df

,tweet,label
0,sip n shop come thru right now marjais popular...,0
1,i dont know about you but my family and i will...,1
2,msignorile immunizations should be mandatory p...,2
3,president obama spoke in favor of vaccination ...,0
4,myfoxla arizona monitoring hundreds for measle...,0
...,...,...
15971,salon if u believe the antivax nutcases caused...,1
15972,how do you feel about parents who dont vaccina...,0
15973,70 preschoolers tested for measles in simi val...,0
15974,finance minister budget offers room to procure...,0


In [ ]:
df2['tweet'] = df2['tweet'].apply(lambda x: tag.sub('', x)) #removing html labels

df2['tweet'] = df2['tweet'].replace(r'http\S+', '', regex=True).replace(r'www.\S+', '', regex=True).replace(r'http\S+', '', regex=True).replace(r'"', '', regex=True)


##Lemmatizer

In [ ]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return " ".join([lemmatizer.lemmatize(w, pos="v") for w in w_tokenizer.tokenize(text)])

df['tweet'] = df.tweet.apply(lemmatize_text).copy()

df

,tweet,label
0,sip n shop come thru right now marjais popular...,0
1,i dont know about you but my family and i will...,1
2,msignorile immunizations should be mandatory p...,2
3,president obama speak in favor of vaccination ...,0
4,myfoxla arizona monitor hundreds for measles l...,0
...,...,...
15971,salon if u believe the antivax nutcases cause ...,1
15972,how do you feel about parent who dont vaccinat...,0
15973,70 preschoolers test for measles in simi valley,0
15974,finance minister budget offer room to procure ...,0


In [ ]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return " ".join([lemmatizer.lemmatize(w, pos="v") for w in w_tokenizer.tokenize(text)])

df2['tweet'] = df2.tweet.apply(lemmatize_text).copy()

df2

,tweet,label
0,user they have a massive surge in with covid d...,1
1,require vaccines for school parent and guardia...,0
2,kcstar two more johnson county children have m...,0
3,nv can do better which state be the best and w...,2
4,nothing like kill ourselves w our own fear mta...,2
...,...,...
2277,rt abc7 number of measles case report in cali ...,0
2278,evidence point to the idea that measles affect...,0
2279,wheres savedyouaclick voxdotcom why you should...,2
2280,some of my favorite people have autism if that...,2


## Create test set 
####Splitted from train set

In [ ]:
train, test = train_test_split(df, test_size=0.125)

In [ ]:
valid = df2

Save the dataframes to files

In [ ]:
train.to_csv("/content/drive/My Drive/train_sdi1700170.csv",index=False)
test.to_csv("/content/drive/My Drive/test_sdi1700170.csv",index=False)
valid.to_csv("/content/drive/My Drive/valid_sdi1700170.csv",index=False)